# Using the Vertex AI PaLM API to Generate and Ideate

# Core code
Let's define some variables that will be used throughout this notebook.

These are the GCP Project ID `project_id`, the Model name `model_name` which is any name you prefer, and finally the Dataset name `dataset_name`.
The dataset needs to exist in the same Project as `project_id` and you'll need appropriate access to create and delete.

## Setup

In [1]:
from typing import Union
import sys

import os
import io
import json
import base64
import requests
import concurrent.futures
import time

import numpy as np
import pandas as pd

import vertexai
import vertexai.language_models as language_models
import vertexai.preview.language_models as language_models_preview
from google.cloud import aiplatform
from google.cloud import documentai
from google.cloud.documentai_v1 import Document
from google.cloud import storage
from google.cloud import bigquery

from IPython.display import display, Markdown, Latex


2023-11-27 11:29:15.876367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
PROJECT_ID = 'mg-ce-demos'
REGION = 'us-central1'


In [3]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)
aiplatform.init(project = PROJECT_ID, location = REGION)


## Ideate

In [4]:
textgen_model_32k = language_models_preview.TextGenerationModel.from_pretrained('text-bison-32k')
textgen_model = language_models.TextGenerationModel.from_pretrained('text-bison@latest')

In [5]:
# Provide some context
pre_text = "Pretend you're a marketing specialist, "

# Ask the LLM
text = "Generate a social media post for a new manufacturing robot product"

prompt_ideate = pre_text+text

In [7]:
display(Markdown('## Prompt:'))
print(prompt_ideate)
display(Markdown('## Response:'))

# Send prompt to LLM
display(Markdown(str(textgen_model.predict(
   (prompt_ideate),
    max_output_tokens=1024,
    temperature=0.4,
    top_p=0.8,
    top_k=40,
).text)))

## Prompt:

Pretend you're a marketing specialist, Generate a social media post for a new manufacturing robot product


## Response:

 Introducing the new manufacturing robot product - the ultimate solution for increased productivity and efficiency in your production line. With its state-of-the-art technology and user-friendly interface, this robot is designed to revolutionise the manufacturing industry. Say goodbye to manual labour and hello to a new era of automated production. #ManufacturingRobot #Automation #Productivity #Efficiency #Manufacturing

## Extract 

In [18]:
# Provide some context
pre_text_extract = """Extract the technical specifications from the text below in a JSON format.

Text: type is Mechanical Joint Pipe, pipe size is 3, pipe thickness from .25 to .4, outside diameter is 3.96, bell weight is 11, bolts gasket weight is 7
JSON: {
  \"product_type\":\"Mechanical Joint Pipe\",
  \"size_in\":\"3\",
  \"thickness_in\": [\".25\", \".4\"],
  \"out_diameter_in\":\"3.96\",
  \"bell_weight_lb\":\"11\",
  \"bolts_gasket_weight_lb\":\"7\"
}

Text: type is Mechanical Joint Pipe, pipe size is 4, pipe thickness from .26 to .41, outside diameter is 4.80, bell weight is 16, bolts gasket weight is 10
JSON: {
  \"product_type\":\"Mechanical Joint Pipe\",
  \"size_in\":\"4\",
  \"thickness_in\": [\".26\", \".41\"],
  \"out_diameter_in\":\"4.8\",
  \"bell_weight_lb\":\"16\",
  \"bolts_gasket_weight_lb\":\"10\"
}

"""

# Ask the LLM
text_extract = """Text: type is Mechanical Joint Pipe, pipe size is 5, bolts gasket weight is 12, outside diameter is 5.7, pipe thickness from .28 to .45, bell weight is 18
JSON:
"""

prompt_extract = pre_text_extract+text_extract
#print(prompt_extract)

In [19]:
display(Markdown(str(textgen_model.predict(
   (prompt_extract),
    max_output_tokens=1024,
    temperature=0.4,
    top_p=0.8,
    top_k=40,
).text)))

 {
  "product_type":"Mechanical Joint Pipe",
  "size_in":"5",
  "thickness_in": [".28", ".45"],
  "out_diameter_in":"5.7",
  "bell_weight_lb":"18",
  "bolts_gasket_weight_lb":"12"
}

## Summarize

## Now we can stick it behind a UI

In [56]:
def extractor_bot(text):
    pre_text = """Extract the technical specifications from the text below in a JSON format.

    Text: type is Mechanical Joint Pipe, pipe size is 3, pipe thickness from .25 to .4, outside diameter is 3.96, bell weight is 11, bolts gasket weight is 7
    JSON: {
      \"product_type\":\"Mechanical Joint Pipe\",
      \"size_in\":\"3\",
      \"thickness_in\": [\".25\", \".4\"],
      \"out_diameter_in\":\"3.96\",
      \"bell_weight_lb\":\"11\",
      \"bolts_gasket_weight_lb\":\"7\"
    }
    
    Text: type is Mechanical Joint Pipe, pipe size is 4, pipe thickness from .26 to .41, outside diameter is 4.80, bell weight is 16, bolts gasket weight is 10
    JSON: {
      \"product_type\":\"Mechanical Joint Pipe\",
      \"size_in\":\"4\",
      \"thickness_in\": [\".26\", \".41\"],
      \"out_diameter_in\":\"4.8\",
      \"bell_weight_lb\":\"16\",
      \"bolts_gasket_weight_lb\":\"10\"
    }

    Text: type is Mechanical Joint Pipe, pipe size is 4, pipe thickness from .26 to .41, bell weight is 16, bolts gasket weight is 10
    JSON: {
      \"product_type\":\"Mechanical Joint Pipe\",
      \"size_in\":\"4\",
      \"thickness_in\": [\".26\", \".41\"],
      \"out_diameter_in\":,
      \"bell_weight_lb\":\"16\",
      \"bolts_gasket_weight_lb\":\"10\"
    }
    
    """
    prompt = f"""{pre_text}
    
    Text: {text}
    JSON:
    """
    
    result = textgen_model.predict(
        prompt,
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    ).text

    result_json = json.loads(result)
    
    return result, result_json

In [ ]:
def marketing_bot(text):
    pre_text = "Pretend you're a marketing specialist, "
    prompt = pre_text + text
    result = textgen_model.predict(
        prompt,
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    )
    
    return prompt, result

In [58]:
import gradio as gr
import json

with gr.Blocks() as demo:
    gr.Markdown(
    """
    ## Pocket Engineer Extractor
    """)
    with gr.Row():
        input_text = gr.Textbox(label="Input Text", value="type is Mechanical Joint Pipe, pipe size is 5, bolts gasket weight is 12, outside diameter is 5.7, pipe thickness from .28 to .45, bell weight is 18")
        
    with gr.Row():
        generate = gr.Button("Generate Response")
        
    with gr.Row():
        label3 = gr.Textbox(label="Response generated by LLM")

    with gr.Row():
        label4 = gr.Textbox(label="JSON output")

    generate.click(extractor_bot, input_text, [label3, label4])
demo.launch(share=False, debug=False)

Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
